In [1]:
import sqlite3
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, precision_recall_curve
import pickle


import os

In [2]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
db = r'C:/Users/jonat/UO_Bootcamp/Group_project/git_Water_Quality_Analysis/Water_Quality_Analysis/Machine_Learning/Binary_Classification/New_Test_Data/Test_Data_DB.db'
# Connect to SQLite database
conn = sqlite3.connect(db)
  
# Create cursor object
cursor = conn.cursor()


df = pd.read_sql_query("SELECT * FROM Census_Data INNER JOIN Summary_Test_data on Census_Data.county_FIPS = Summary_Test_data.county_FIPS",conn)

In [8]:
df.sample(10)

,county_FIPS,Geographic_Area_Name,County,GEOID,Total_Population,White,Black,Native,Asian,Pacific_Islander,...,Shannon_Race_DI,Shannon_Ethnic_DI,Gini_Index,county_FIPS,Num_Contaminants,Sum_Population_Served,Sum_ContaminantFactor,Min_Contaminant_Factor,Max_Contaminant_Factor,Avg_Contaminant_Factor
19,5075,"Lawrence County, Arkansas",Lawrence County,0500000US05075,16216,15044,144,53,40,13,...,0.426299,0.105782,0.4376,05075,1,663,20,20,20,20.00
22,5085,"Lonoke County, Arkansas",Lonoke County,0500000US05085,74015,61624,4387,369,734,51,...,0.813348,0.194889,0.4197,05085,6,115412,6704,474,1855,1117.33
7,5035,"Crittenden County, Arkansas",Crittenden County,0500000US05035,48163,19276,25906,137,328,18,...,1.042171,0.133521,0.4694,05035,1,540,37,37,37,37.00
3,5023,"Cleburne County, Arkansas",Cleburne County,0500000US05023,24711,22953,70,134,103,12,...,0.425622,0.119009,0.4465,05023,1,11150,1212,1212,1212,1212.00
13,5057,"Hempstead County, Arkansas",Hempstead County,0500000US05057,20065,11006,5684,141,97,6,...,1.428771,0.417772,0.4408,05057,1,150,0,0,0,0.00
40,5147,"Woodruff County, Arkansas",Woodruff County,0500000US05147,6269,4255,1614,9,23,1,...,0.900231,0.079191,0.4993,05147,2,2534,516,241,275,258.00
4,5027,"Columbia County, Arkansas",Columbia County,0500000US05027,22801,13371,7867,75,169,1,...,1.053814,0.148055,0.5365,05027,2,1065,1153,212,941,576.50
28,5103,"Ouachita County, Arkansas",Ouachita County,0500000US05103,22650,12194,8900,72,127,4,...,1.031241,0.108675,0.4474,05103,6,19038,5711,29,1841,951.83
6,5031,"Craighead County, Arkansas",Craighead County,0500000US05031,111231,80639,18474,357,1693,62,...,1.061992,0.228280,0.4953,05031,2,2417,105,0,105,52.50
25,5093,"Mississippi County, Arkansas",Mississippi County,0500000US05093,40685,23208,14324,100,246,9,...,1.101316,0.179003,0.4719,05093,2,3856,61,25,36,30.50


In [9]:
df.columns

Index(['county_FIPS', 'Geographic_Area_Name', 'County', 'GEOID',
       'Total_Population', 'White', 'Black', 'Native', 'Asian',
       'Pacific_Islander', 'Other', 'Two_or_more_Races', 'Hispanic',
       'Not_Hispanic', 'Not_White', 'pct_White', 'pct_Black', 'pct_Native',
       'pct_Asian', 'pct_Pacific_Islander', 'pct_Other', 'pct_Not_White',
       'pct_Hispanic', 'pct_Not_Hispanic', 'pct_Two_or_more_Races',
       'Simpson_Race_DI', 'Simpson_Ethnic_DI', 'Shannon_Race_DI',
       'Shannon_Ethnic_DI', 'Gini_Index', 'county_FIPS', 'Num_Contaminants',
       'Sum_Population_Served', 'Sum_ContaminantFactor',
       'Min_Contaminant_Factor', 'Max_Contaminant_Factor',
       'Avg_Contaminant_Factor'],
      dtype='object')

In [25]:
df_model = df[['Simpson_Race_DI', 'Simpson_Ethnic_DI', 'Shannon_Race_DI',
       'Shannon_Ethnic_DI', 'Gini_Index', 'Num_Contaminants',
       'Sum_ContaminantFactor', 'Avg_Contaminant_Factor']]

mdf = df_model.drop(columns=['Shannon_Ethnic_DI', 'Simpson_Race_DI'])
mdf.sample(41)

,Simpson_Ethnic_DI,Shannon_Race_DI,Gini_Index,Num_Contaminants,Sum_ContaminantFactor,Avg_Contaminant_Factor
25,0.083272,1.101316,0.4719,2,61,30.50
20,0.065711,0.972260,0.4403,1,553,553.00
37,0.061426,0.488795,0.4248,3,2005,668.33
31,0.034407,0.647278,0.4374,1,805,805.00
27,0.022444,0.383237,0.4132,1,213,213.00
0,0.053777,0.894256,0.4214,1,67,67.00
1,0.071427,1.000107,0.4646,1,21,21.00
28,0.044527,1.031241,0.4474,6,5711,951.83
6,0.113641,1.061992,0.4953,2,105,52.50
10,0.072765,1.027685,0.5068,1,1142,1142.00


In [24]:
mdf.describe()

,Simpson_Ethnic_DI,Shannon_Race_DI,Gini_Index,Num_Contaminants,Sum_ContaminantFactor,Avg_Contaminant_Factor
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,0.096745,0.874314,0.457724,2.365854,1566.804878,555.738780
std,0.086054,0.316288,0.028188,1.867567,1882.770807,460.092062
min,0.022444,0.365466,0.405300,1.000000,0.000000,0.000000
25%,0.046873,0.595514,0.437900,1.000000,114.000000,67.000000
50%,0.065508,0.894256,0.454100,2.000000,805.000000,558.000000
75%,0.113641,1.053814,0.475400,3.000000,2310.000000,995.200000
max,0.453668,1.550986,0.536500,9.000000,6704.000000,1519.000000


In [14]:
# Create our features
X = mdf

In [15]:
# define standard scaler
scaler = StandardScaler()
# transform data
X_scaled = scaler.fit_transform(X)

In [19]:
# load the model from disk
filename = 'EEC_model.sav'
eec = pickle.load(open(filename, 'rb'))
y_pred = eec.predict(X_scaled)
print(y_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]


In [ ]:
filename = 'BRFC_model.sav'

In [20]:
# load the model from disk
filename = 'BRFC_model.sav'
brfc = pickle.load(open(filename, 'rb'))
y_pred = brfc.predict(X_scaled)
print(y_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]


In [21]:
len(y_pred)

41

In [23]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)